# Forecasting Product Demand

In this example, we will be forecasting the monthly demand of different products.  The data used is a public Kaggle dataset consisting of order demand per product and warehouse: [Product Demand Data](https://www.kaggle.com/felixzhao/productdemandforecasting).

We will be using Sparkling Water to ingest the data and add historical lags.

Our Machine Learning Workflow is: 

1. Import data into Spark
2. Exploratory Analysis
3. Feature engineering
   * Add time lag columns
4. Train a baseline model
5. Train model with lag features
6. Train model with additional features
7. Compare models
8. Shut down sparkling water cluster

# Step 1 (of 8).  Import data into Spark

In [ ]:
# Initiate H2OContext on top of Spark

from pysparkling import *
hc = H2OContext.getOrCreate(spark)

In [ ]:
# Import data

from pyspark.sql.types import *

schema = StructType([StructField("Product_Code", StringType(), True),
                     StructField("Warehouse", StringType(), True),
                     StructField("Product_Category", StringType(), True),
                     StructField("Month", DateType(), True),
                     StructField("Total_Demand", DoubleType(), True),
                     StructField("Number_Orders", DoubleType(), True)])

# https://s3-us-west-2.amazonaws.com/h2o-tutorials/data/topics/time_series/product_demand/Monthly_Product_Demand.csv
products_df = spark.read.csv("../../data/topics/time_series/product_demand/Monthly_Product_Demand.csv", header = True, schema = schema)

In [ ]:
products_df.show(5)

# Step 2 (of 8).  Exploratory Analysis

We will start by exploring and analyzing our data.  We will first explore the data in PySpark and then do the same exploration in H2O. 

### PySpark Exploratory Analysis

In [ ]:
# Data Schema
products_df.printSchema()

In [ ]:
# Dimensions
print("Number of rows: ", "{:,}".format(products_df.count()))
print("Number of columns: ", "{:,}".format(len(products_df.columns)))

In [ ]:
# Distinct Products
print("Number of Products: ", products_df.select("Product_Code").distinct().count())

In [ ]:
# Distinct Warehouses
print("Number of Warehouses: ", products_df.select("Warehouse").distinct().count())

In [ ]:
# Range of Demand
min_demand = products_df.agg({"Total_Demand": "min"}).collect()[0][0]
max_demand = products_df.agg({"Total_Demand": "max"}).collect()[0][0]
print("Demand Range: ", min_demand, " to ", max_demand)

In [ ]:
# Date Range
min_date = products_df.agg({"Month": "min"}).collect()[0][0]
max_date = products_df.agg({"Month": "max"}).collect()[0][0]
print("Date Range: ", min_date, " to ", max_date)

In [ ]:
# Number of records per product and warehouse
from pyspark.sql.functions import count, col 
cnts = products_df.groupBy("Product_Code", "Product_Category", "Warehouse").agg(count("*").alias("cnt")).alias("cnts")
cnts.show()

In [ ]:
# Plot some of the time series data
plot_data = products_df.where((products_df.Product_Code == "Product_1846"))
plot_data.count()

In [ ]:
plot_data = plot_data.toPandas()

In [ ]:
plot_data.head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,1);
plot_data.groupby("Warehouse").plot(x="Month", y="Total_Demand", ax=ax);
plt.legend([v[0] for v in plot_data.groupby('Warehouse')['Warehouse']])

### H2O Exploratory Analysis

We will repeat the same exploratory analysis on the H2O Frame to show the differences in functionality.

In [ ]:
## Convert Spark DataFrame to H2O Frame

import h2o
products_hf = hc.as_h2o_frame(products_df, "productsTable")

In [ ]:
# Data Schema
products_hf.describe()

In [ ]:
# Convert String to Categorical
cat_cols = ["Product_Code", "Product_Category", "Warehouse"]
for i in cat_cols:
    products_hf[i] = products_hf[i].asfactor()

In [ ]:
# Dimensions
print("Number of rows: ", "{:,}".format(products_hf.nrow))
print("Number of columns: ", "{:,}".format(products_hf.ncol))

In [ ]:
# Number of unique products
print("Number of Products: ", products_hf["Product_Code"].table().nrow)

In [ ]:
# Number of unique warehouses
print("Number of Warehouses: ", products_hf["Warehouse"].table().nrow)

In [ ]:
# Range of Demand
products_hf["Total_Demand"].hist()

In [ ]:
# Date Range
print("Date Range")
products_hf = products_hf.sort("Month")
print(products_hf["Month"][[0, (products_hf.nrow - 1)], "Month"])

In [ ]:
# Count per Product and Warehouse
print("Count per Product and Warehouse")
cnts = products_hf.group_by(["Product_Code", "Product_Category", "Warehouse"]).count().get_frame()
cnts.head()

# Step 3 (of 8).  Feature Engineering

We will add new features to our data that can help predict the Demand for a given product and warehouse.  Features that tell us:
* what was the Demand for a product and warehouse last month, two months ago, three months ago, etc?  
* what was the Number of Order for a product and warehouse last month?

can be very predictive in forecasting.  To create these features we will use PySpark's window function.

In [ ]:
## Add Total Demand from the Previous month, Previous 2 months, Previous 3 months, etc per Product and Warehouse
from pyspark.sql.functions import lag, col
from pyspark.sql.window import Window

w = Window().partitionBy([col(x) for x in ["Product_Code", "Product_Category", "Warehouse"]]).orderBy(col("Month"))
ext_products_df = products_df.select("*", lag("Total_Demand", count = 1).over(w).alias("Demand_lag1"),
                                     lag("Total_Demand", count = 2).over(w).alias("Demand_lag2"),
                                     lag("Total_Demand", count = 3).over(w).alias("Demand_lag3"),
                                     lag("Total_Demand", count = 4).over(w).alias("Demand_lag4"),
                                     lag("Total_Demand", count = 5).over(w).alias("Demand_lag5"),
                                     lag("Total_Demand", count = 6).over(w).alias("Demand_lag6"),
                                     lag("Total_Demand", count = 12).over(w).alias("Demand_lag12")).na.drop()

In [ ]:
ext_products_df.show()

In [ ]:
## Add Number of Orders for Previous month per Product and Warehouse

ext_products_df = ext_products_df.select("*", lag("Number_Orders", count = 1).over(w).alias("Number_Orders_lag1")).na.drop()

In [ ]:
## Convert Spark DataFrame to H2O Frame
ext_products_hf = hc.as_h2o_frame(ext_products_df, "productsWithLagsTable")

In [ ]:
# Convert String to Categorical
for i in cat_cols:
    ext_products_hf[i] = ext_products_hf[i].asfactor()

# Step 4 (of 8).  Train baseline model

We will begin our machine learning section by training a model using solely our original data.  We will use this model as a baseline to compare the performance of new models with additional features.

In [ ]:
# Set Predictors
predictors = ["Product_Code", "Warehouse", "Product_Category"]
response = "Total_Demand"

In [ ]:
# Split data into training and testing by time

train = ext_products_hf[ext_products_hf["Month"].year() < 2015]
test = ext_products_hf[ext_products_hf["Month"].year() == 2015]

In [ ]:
# Train GBM Model with Early Stopping

from h2o.estimators import H2OGradientBoostingEstimator
baseline_model = H2OGradientBoostingEstimator(model_id = "baseline_model.hex",
                                              stopping_rounds = 3, stopping_metric = "MAE",
                                              score_tree_interval = 10, ntrees = 500)
baseline_model.train(x = predictors,
                     y = response,
                     training_frame = train,
                     validation_frame = test)

In [ ]:
baseline_model.plot()

In [ ]:
print("Baseline - MAE: " + "{:,}".format(round(baseline_model.mae(valid = True))))

# Step 5 (of 8).  Train model with lags

We will see if we can improve our model by adding our lag features as new predictors.

In [ ]:
# Add Lag Predictors
ext_predictors = list(set(train.col_names) - set(["Total_Demand", "Number_Orders"]))
ext_predictors

In [ ]:
# Train GBM Model with Early Stopping

lag_features_v1 = H2OGradientBoostingEstimator(model_id = "lag_features_v1.hex",
                                              stopping_rounds = 3,  stopping_metric = "MAE",
                                              score_tree_interval = 10, ntrees = 500)
lag_features_v1.train(x = ext_predictors,
                      y = response,
                      training_frame = train,
                      validation_frame = test)

In [ ]:
lag_features_v1.plot()

In [ ]:
print("Baseline - MAE: " + "{:,}".format(round(baseline_model.mae(valid = True))))
print("Lag Features - MAE: " + "{:,}".format(round(lag_features_v1.mae(valid = True))))

The graph below shows the variable importance for the gradient boosted model.  The most important predictors are the demand lags.  We can use the partial dependency plots to see the relationship between these features and the model's prediction.

In [ ]:
%matplotlib inline
lag_features_v1.varimp_plot(num_of_features = 10)

In [ ]:
# Filter data to common demand
max_demand = train["Total_Demand"].quantile(prob = [0.9])[0, 1]
pdp_data = train[(train["Demand_lag3"] < max_demand) & (train["Demand_lag3"] >= 0)]
# create pdp's
pdps = lag_features_v1.partial_plot(data = pdp_data, cols = ["Demand_lag3"])

The partial plots show that the Demand from last quarter is related to the current month Demand.

# Step 6 (of 8).  Train Model with Additional Features

We will now use H2O to add additional features such as the change in demand over days.

A feature we would like to add is the average demand for product, warehouse, and product & warehouse.  Using the actual demand when calculating the average, however, can result in data leakage.  Instead we will calculate the average of the previous month's Demand.

In [ ]:
# Average Demand Lag 1 by Product & Warehouse
avg_demand_group = ext_products_hf.group_by(["Product_Code", "Product_Category", "Warehouse"]).mean(["Demand_lag1"])
avg_demand_group = avg_demand_group.get_frame()
avg_demand_group.col_names = ["Product_Code", "Product_Category", "Warehouse", "mean_Demand_lag1_Product_Warehouse"]

In [ ]:
avg_demand_group.head()

In [ ]:
# Average Demand Lag 1 by Warehouse
avg_demand_warehouse = ext_products_hf.group_by(["Warehouse"]).mean(["Demand_lag1"]).get_frame()
avg_demand_warehouse.col_names = ["Warehouse", "mean_Demand_lag1_Warehouse"]
avg_demand_warehouse.head()

In [ ]:
# Average Demand Lag 1 by Product
avg_demand_product = ext_products_hf.group_by(["Product_Code"]).mean(["Demand_lag1"]).get_frame()
avg_demand_product.col_names = ["Product_Code", "mean_Demand_lag1_Product"]
avg_demand_product.head()

In [ ]:
# Average Demand Lag 1 by Product Category
avg_demand_product_cat = ext_products_hf.group_by(["Product_Category"]).mean(["Demand_lag1"]).get_frame()
avg_demand_product_cat.col_names = ["Product_Category", "mean_Demand_lag1_ProductCat"]
avg_demand_product_cat.head()

In [ ]:
# Merge averages to our original frame
ext_products_hf = ext_products_hf.merge(avg_demand_group, all_x = True, all_y = False)
ext_products_hf = ext_products_hf.merge(avg_demand_warehouse, all_x = True, all_y = False)
ext_products_hf = ext_products_hf.merge(avg_demand_product, all_x = True, all_y = False)
ext_products_hf = ext_products_hf.merge(avg_demand_product_cat, all_x = True, all_y = False)

In [ ]:
ext_products_hf.head()

In [ ]:
# Extract the month in case there is seasonality 
ext_products_hf["Categorical_Month"] = ext_products_hf["Month"].month().asfactor()
ext_products_hf["Categorical_Month"].table()

In [ ]:
# Add Lag Predictors
ext_predictors = list(set(ext_products_hf.col_names) - set(["Total_Demand", "Number_Orders"]))
ext_predictors

In [ ]:
# Split data into training and testing by time

train = ext_products_hf[ext_products_hf["Month"].year() < 2015]
test = ext_products_hf[ext_products_hf["Month"].year() == 2015]

In [ ]:
# Train GBM Model with Early Stopping

lag_features_v2 = H2OGradientBoostingEstimator(model_id = "lag_features_v2.hex",
                                              stopping_rounds = 3,  stopping_metric = "MAE",
                                              score_tree_interval = 10, ntrees = 500)
lag_features_v2.train(x = ext_predictors,
                      y = response,
                      training_frame = train,
                      validation_frame = test)

# Step 7 (of 8).  Compare Models

Below we compare our models by the Mean Absolute Error.

In [ ]:
print("Baseline - MAE: " + "{:,}".format(round(baseline_model.mae(valid = True))))
print("Lag Features V1 - MAE: " + "{:,}".format(round(lag_features_v1.mae(valid = True))))
print("Lag Features V2 - MAE: " + "{:,}".format(round(lag_features_v2.mae(valid = True))))

In [ ]:
lag_features_v2.varimp_plot()

# Step 8 (of 8).  Shutdown the Sparkling Water Cluster

In [ ]:
# Stop H2O and Spark services
h2o.cluster().shutdown()
spark.stop()

# Bonus: Github location for this tutorial

* https://github.com/h2oai/h2o-tutorials/tree/master/training/sparkling_water_hands_on/time_series